In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import os
import logging
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Setup for Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (without opening browser window)
chrome_driver_path = os.getenv('CHROME_DRIVER_PATH')  # Path to chromedriver from .env file

# Function to check if the ticker has numbers
def has_num(shifra):
    return any(char.isdigit() for char in shifra)

# Function to fetch tickers using Selenium
def fetch_tikeri_selenium():
    # Initialize WebDriver
    service = Service(chrome_driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        url = 'https://www.mse.mk/mk/stats/symbolhistory/kmb'
        driver.get(url)
        print(f"Page URL: {driver.current_url}")  # Check if the page is loaded

        # Wait for the dropdown menu to be available
        wait = WebDriverWait(driver, 10)
        dropdown = wait.until(EC.presence_of_element_located((By.ID, "Code")))

        # Extract all option elements from the dropdown
        options = dropdown.find_elements(By.TAG_NAME, 'option')

        if not options:
            print("No options found in dropdown!")

        tikeri = [option.get_attribute('value') for option in options if not has_num(option.get_attribute('value'))]

        print(f"Extracted Tickers: {tikeri}")  # Print the extracted tickers

        return tikeri

    except Exception as e:
        logging.error(f"Error fetching ticker data: {e}")
        return []

    finally:
        driver.quit()

# Function to save tickers to a pandas DataFrame and to a CSV file
def save_to_dataframe(shifri_list, filename='tikeri.csv'):
    try:
        print(f"Saving {len(shifri_list)} tickers to {filename}...")  # Check if the function is called

        # Check if the CSV file already exists
        if os.path.exists(filename):
            df_existing = pd.read_csv(filename)
            print(f"Loaded existing data from {filename}, {len(df_existing)} records found.")
        else:
            df_existing = pd.DataFrame(columns=['Ticker'])
            print("No existing file found, creating a new one.")

        # Create a new DataFrame for the new data
        df_new = pd.DataFrame(shifri_list, columns=['Ticker'])
        print(f"New tickers DataFrame: {df_new}")

        # Concatenate the old and new DataFrames and drop duplicates
        df_all = pd.concat([df_existing, df_new]).drop_duplicates().reset_index(drop=True)

        # Save the updated DataFrame back to CSV
        df_all.to_csv(filename, index=False)
        print(f"Data saved to {filename}")

    except Exception as e:
        logging.error(f"Error saving data to DataFrame: {e}")

# Initialization function to call the other functions
def init():
    tikeri = fetch_tikeri_selenium()
    if tikeri:
        print(f"Extracted tickers: {tikeri}")
    else:
        print("No tickers extracted.")
    save_to_dataframe(tikeri)
    return tikeri

# Start the process
if __name__ == "__main__":
    logging.basicConfig(level=logging.INFO)  # Enable logging
    init()

Page URL: https://www.mse.mk/mk/stats/symbolhistory/kmb
Extracted Tickers: ['ADIN', 'ALK', 'ALKB', 'AMEH', 'APTK', 'ATPP', 'AUMK', 'BANA', 'BGOR', 'BIKF', 'BIM', 'BLTU', 'CBNG', 'CDHV', 'CEVI', 'CKB', 'CKBKO', 'DEBA', 'DIMI', 'EDST', 'ELMA', 'ELNC', 'ENER', 'ENSA', 'EUHA', 'EUMK', 'EVRO', 'FAKM', 'FERS', 'FKTL', 'FROT', 'FUBT', 'GALE', 'GDKM', 'GECK', 'GECT', 'GIMS', 'GRDN', 'GRNT', 'GRSN', 'GRZD', 'GTC', 'GTRG', 'IJUG', 'INB', 'INHO', 'INOV', 'INPR', 'INTP', 'JAKO', 'JUSK', 'KARO', 'KDFO', 'KJUBI', 'KKST', 'KLST', 'KMB', 'KMPR', 'KOMU', 'KONF', 'KONZ', 'KORZ', 'KPSS', 'KULT', 'KVAS', 'LAJO', 'LHND', 'LOTO', 'LOZP', 'MAGP', 'MAKP', 'MAKS', 'MB', 'MERM', 'MKSD', 'MLKR', 'MODA', 'MPOL', 'MPT', 'MPTE', 'MTUR', 'MZHE', 'MZPU', 'NEME', 'NOSK', 'OBPP', 'OILK', 'OKTA', 'OMOS', 'OPFO', 'OPTK', 'ORAN', 'OSPO', 'OTEK', 'PELK', 'PGGV', 'PKB', 'POPK', 'PPIV', 'PROD', 'PROT', 'PTRS', 'RADE', 'REPL', 'RIMI', 'RINS', 'RZEK', 'RZIT', 'RZIZ', 'RZLE', 'RZLV', 'RZTK', 'RZUG', 'RZUS', 'SBT', 'SDOM', 'SIL'